# Fake zaposlenici fakulteta #

U ovom notebooku cemo pokazati kako napraviti laznu listu zaposlenika nekog fakulteta. 

Rijesenje ovog zadatka je proizvedeni JSON file takav da on predstavlja listu zaposlenika. Svaki clan liste je dictionary koji ima kljuceve:

* ime
* radnoMjesto
* placa
* kolegiji

Ovaj JSON mora zadovoljavati neka ogranicenja:

1. postoje 6 vrsta zaposlenika: dekan, redProf, izvProf, doc, asist, i admin
2. oko 30% zaposlenika je administracija (za nase potrebe, dekan nema nastavne obveze i ubrajamo ga u administraciju)
3. postoji jedan dekan i dekan ima najvislju placu, koja je 5% veca od sljedece najvislje place
4. distribucija znanstveno-nastavnih i suradnickih zvanja (nazivat cemo ih "nastavnici") je uniformna
5. place su: redProf= 12000 +/- 10%, izvProf= 10000 +/- 10%, doc= 8000 +/- 10%, asist= 6000 +/- 10%, admin= 8000 +/- 30% . Varijacije su uniformne (odnosno koristite **randint**)
6. broj kolegija je nacelno neogranicen, njihova imena su kolegij1,kolegij2, kolegij3... U JSONu, svaki nastavnik ima listu svojih **cetiri** kolegija. Administracija i dekan imaju praznu listu kolegija.
7. ukupan broj zaposlenika je 150


Prvi korak je da napravimo funkciju koja ce generitati placu oko odredene "osnove" (probajte nekoliko puta pokrenuti sljedecu celiju):


In [ ]:
from random import randint
def izrcPlace(osnova, varijacija=0.1):
    return randint(osnova-(osnova*varijacija),osnova+(osnova*varijacija))

print(izrcPlace(10000))
print(izrcPlace(10000, 0.5))

Sljedece cemo inicijalizirati vrijednosti i liste koje nam trebaju i koje cemo onda spajati:

In [ ]:
brojZaposlenika = 150
listaImena = ["osoba"+str(x) for x in range(1,brojZaposlenika+1)]

listaAdmin = ["admin" for x in range(1, (round(brojZaposlenika/3)))]
print(len(listaAdmin)) # neka ostane 49 radi dekana

nastavnici = ["redProf","izvProf","doc","asist"]
brojNastavnika = brojZaposlenika-len(listaAdmin)-1

listaRedProf = ["redProf" for x in range(1,round(brojNastavnika/4)+1)]
listaIzvProf = ["izvProf" for x in range(1,round(brojNastavnika/4)+1)]
listaDoc = ["doc" for x in range(1,round(brojNastavnika/4)+1)]
listaAsist = ["asist" for x in range(1,round(brojNastavnika/4)+1)]
print(len(listaRedProf), len(listaIzvProf), len(listaDoc), len(listaAsist))

Sada mozemo poceti sastavljati JSON. Sastavit cemo dvije liste, jednu za nastavnike, a drugu za administraciju i dekana. Prvo krecemo s nastavnicima, jer ce nam trebati najveca placa da izracunamo placu dekana. Kada je prva verzija gotova, tada cemo dodati kolegije. Za sada svi dobivaju praznu listu za kolegije. 

Prvo nam treba dovoljno kolegija, sigurno nam je dovoljno njih 1000:

In [ ]:
listaKolegija = ["kolegij"+str(x) for x in range(1,1001)]

Sada radimo prvi dio JSONa, sa nastavnicima:

In [ ]:
from random import shuffle
gotovo_Red_Izv_Doc = []
listaZnNastavnici = listaRedProf + listaIzvProf + listaDoc + listaAsist
shuffle(listaZnNastavnici)
#print(listaZnNastavnici)
counter=0
for i in listaZnNastavnici:
    #print(i)
    novi={}
    novi["radnoMjesto"]=i
    if i=="redProf":
        mark=12000
    elif i=="izvProf":
        mark=10000
    elif i=="doc":
        mark=8000
    elif i=="asist":
        mark=6000
    novi["placa"]= izrcPlace(mark)
    novi["kolegiji"]=[listaKolegija[counter],listaKolegija[counter+1],listaKolegija[counter+2],listaKolegija[counter+3]]
    counter=counter+4
    gotovo_Red_Izv_Doc.append(novi)
print(gotovo_Red_Izv_Doc)
    

Sada trebamo napraviti isto za administraciju (dekana cemo rucno dodati u sljedecoj celiji):

In [ ]:
from random import shuffle
gotovo_Admin = []
shuffle(listaAdmin)
print(len(listaAdmin))
for i in listaAdmin:
    #print(i)
    novi={}
    novi["radnoMjesto"]=i
    mark=8000    
    novi["placa"]= izrcPlace(mark,0.3)
    novi["kolegiji"]=[]
    gotovo_Admin.append(novi)
print(gotovo_Admin)

Sada rucno dodajem dekana, ali pazimo na izracun place:

In [ ]:
skoroGotovo=gotovo_Red_Izv_Doc+gotovo_Admin
place=[x["placa"] for x in skoroGotovo]
najvecaPlaca=max(place)
print(najvecaPlaca)
placaDekana=najvecaPlaca+0.05*najvecaPlaca
dekan=[{"radnoMjesto":"dekan", "placa":placaDekana, "kolegiji":[]}]
print(dekan)
gotovo=skoroGotovo+dekan

Sada spajamo sve u jednu listu, i dopunjujemo s imenima

In [ ]:
counter=0
for i in gotovo:
    i["ime"]=listaImena[counter]
    counter=counter+1
print(gotovo[:5])    

I na kraju, cijelu listu spremamo u JSON file:

In [ ]:
import json
Datoteka=open("faukltet.json","w")
sadrzaj=str(json.dumps(gotovo))
print(sadrzaj)
Datoteka.writelines(sadrzaj)
Datoteka.close()